# Liberar DNI - Notebook de pruebas

Este notebook requiere las credenciales de tu DNI para iniciar sesión en el portal de cita previa en un archivo `.env`.
Simplemente crea una copia de `.env.sample` llamada `.env` y rellena los datos con los que figuran en tu DNI.

In [ ]:
import os

from PIL import Image

import pandas as pd

from dotenv import load_dotenv
from pytesseract import image_to_string
from selenium.webdriver import Firefox
from tqdm.auto import tqdm

from utils import (download_captcha_img, extraer_datos_unidad, extract_hours,
                   get_remaining_days, get_remaining_months)

In [ ]:
load_dotenv()

In [ ]:
URL_LOGIN = 'https://www.citapreviadnie.es/citaPreviaDniExp/InicioDNINIE.action'
URL_DNI = 'https://www.citapreviadnie.es/citaPreviaDniExp/InicioTramite.action?idDocumento=D'
URL_PASAPORTE = 'https://www.citapreviadnie.es/citaPreviaDniExp/InicioTramite.action?idDocumento=P'

driver = Firefox()

driver.get(URL_LOGIN)

## Repeat these steps until login works

In [ ]:
download_captcha_img(driver)

In [ ]:
captcha_img = Image.open('captcha.jpg')
solved_captcha = image_to_string(captcha_img).replace(' ', '')[:4]

In [ ]:
solved_captcha

In [ ]:
NUM_DOCUMENTO = os.environ['NUM_DOCUMENTO']
LETRA_DOCUMENTO = os.environ['LETRA_DOCUMENTO']
CODIGO_EQUIPO = os.environ['CODIGO_EQUIPO']
FECHA_VALIDEZ = os.environ['FECHA_VALIDEZ']

inputs = {
    'numDocumento': NUM_DOCUMENTO,
    'letraDocumento': LETRA_DOCUMENTO,
    'codEquipo': CODIGO_EQUIPO,
    'fechaValidez': FECHA_VALIDEZ,
    'codSeguridad': solved_captcha,
}

for k,v in inputs.items():
    elem = driver.find_element_by_id(k)
    elem.send_keys(v)

In [ ]:
submit_button = driver.find_element_by_xpath('/html/body/div/div[4]/div[1]/form/p[6]/input[1]')
submit_button.click()

# DNI Extract

## Provinces

In [ ]:
driver.get(URL_DNI)

In [ ]:
provinces = {}

map_elem = driver.find_element_by_id('Map')
for map_child in map_elem.find_elements_by_css_selector('*'):
    name = map_child.get_attribute('alt')
    url = map_child.get_attribute('href')
    
    provinces[name] = url

## Unidades

In [ ]:
province_name = 'A Coruña'
province_url = provinces[province_name]

driver.get(province_url)

In [ ]:
listas = driver.find_elements_by_class_name('lista')
unidades = [extraer_datos_unidad(ud, province_name) for lista in listas for ud in lista.find_elements_by_tag_name('div')]

In [ ]:
unidad = unidades[0]
unidad_name = unidad['name']
unidad

## Horas

El comportamiento aquí siempre es igual: el mes y día seleccionados (por defecto, el primero) no tienen enlace, los siguientes sí.

In [ ]:
driver.get(unidad['url'])

First try current month, then any remaining months

In [ ]:
citas = extract_hours(driver, province_name, unidad_name)

month_days = get_remaining_days(driver)
for date, date_url in tqdm(month_days.items(), desc='Current month'):
    driver.get(date_url)
    citas.extend(extract_hours(driver, province_name, unidad_name))
    
next_months = get_remaining_months(driver)
for month, month_url in next_months.items():
    driver.get(month_url)
    month_days = get_remaining_days(driver)
    for date, date_url in tqdm(month_days.items(), desc=month):
        driver.get(date_url)
        citas.extend(extract_hours(driver, province_name, unidad_name))

In [ ]:
citas_df = pd.DataFrame(citas)
citas_df.head()

In [ ]:
citas_df.to_csv('citas_coruna.csv', index=False)